In [530]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

In [531]:
vwce = pd.DataFrame(yf.download('VWCE.MI'))
vwce.columns = vwce.columns.droplevel('Ticker')
vwce.reset_index(inplace=True)
vwce.rename(columns={"Price": "Index"}, inplace=True)
print(vwce.columns)
vwce

[*********************100%***********************]  1 of 1 completed

Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')


Price,Date,Adj Close,Close,High,Low,Open,Volume
0,2020-01-14 00:00:00+00:00,78.620003,78.620003,78.750000,78.750000,78.750000,1
1,2020-01-15 00:00:00+00:00,78.570000,78.570000,78.680000,78.500000,78.599998,201
2,2020-01-16 00:00:00+00:00,78.870003,78.870003,78.900002,78.779999,78.900002,717
3,2020-01-17 00:00:00+00:00,79.540001,79.540001,79.660004,79.570000,79.570000,51
4,2020-01-20 00:00:00+00:00,79.529999,79.529999,79.599998,79.599998,79.599998,25
...,...,...,...,...,...,...,...
1250,2024-12-04 00:00:00+00:00,136.039993,136.039993,136.630005,135.750000,135.960007,16392
1251,2024-12-05 00:00:00+00:00,135.960007,135.960007,136.380005,135.729996,136.130005,14158
1252,2024-12-06 00:00:00+00:00,136.020004,136.020004,136.330002,135.500000,135.619995,11645
1253,2024-12-09 00:00:00+00:00,135.940002,135.940002,136.500000,135.600006,136.429993,25078


In [532]:
df=pd.DataFrame(columns=["Date","Close","High","Low","DD%"])
df["Date"]=vwce["Date"]
df["Close"]=vwce["Close"]
df["High"]=vwce["High"]
df["Low"]=vwce["Low"]

df

,Date,Close,High,Low,DD%
0,2020-01-14 00:00:00+00:00,78.620003,78.750000,78.750000,NaN
1,2020-01-15 00:00:00+00:00,78.570000,78.680000,78.500000,NaN
2,2020-01-16 00:00:00+00:00,78.870003,78.900002,78.779999,NaN
3,2020-01-17 00:00:00+00:00,79.540001,79.660004,79.570000,NaN
4,2020-01-20 00:00:00+00:00,79.529999,79.599998,79.599998,NaN
...,...,...,...,...,...
1250,2024-12-04 00:00:00+00:00,136.039993,136.630005,135.750000,NaN
1251,2024-12-05 00:00:00+00:00,135.960007,136.380005,135.729996,NaN
1252,2024-12-06 00:00:00+00:00,136.020004,136.330002,135.500000,NaN
1253,2024-12-09 00:00:00+00:00,135.940002,136.500000,135.600006,NaN


In [533]:
#Defining needed variables
thresholds = np.arange(0, -55, -5)
buffer_percentages = np.array([0.0 ,0.30, 0.20, 0.10, 0.05, 0.05, 0.20, 0.30, 0.40, 0.50, 0.50])
thresholds_dict = dict(zip(thresholds, buffer_percentages))
monthly_investment=500
number_of_stocks=0
buffer_value=0
pmc=0

print(thresholds)

[  0  -5 -10 -15 -20 -25 -30 -35 -40 -45 -50]


In [534]:
def simulation_padd(df):
    maximum = 0
    last_threshold_alert=0
    global pmc
    global number_of_stocks
    global buffer_value
    for i in range(0,len(df)):
        #All'inizio del mese acquisto un'azione, aggiorno il PMC e il buffer
        if df.iloc[i]["Date"].month != df.iloc[i-1]["Date"].month:
            pmc = ((pmc * number_of_stocks) + (df.loc[i,"Close"] * 1))/(number_of_stocks+1)
            buffer_value = buffer_value + (monthly_investment - df.loc[i,"Close"])
            number_of_stocks+=1
            print("PMC: "+str(pmc))
            print("Buffer: "+str(buffer_value))
            print("Numero di azioni: "+str(number_of_stocks))
        #Calcolo il drawdown
        if df.loc[i,"High"] >= maximum:
            maximum = df.loc[i,"High"]
            last_threshold_alert = 0
            dd = 0.0
            df.loc[i,"DD%"] = dd
        else:
            dd = ((df.loc[i,"Close"] - maximum) / maximum)*100
            df.loc[i,"DD%"] = dd.round(2)
        #Se il drawdown è maggiore di una delle soglie investi con una percentuale del buffer
        for i in range(len(thresholds) - 1):
            if thresholds[i] >= dd > thresholds[i + 1]:
                print(f"{dd} è compreso tra {thresholds[i + 1]} e {thresholds[i]}")
                break
        if thresholds[i] < last_threshold_alert:
            print("### ALERT ### "+str(thresholds[i]))
            last_threshold_alert = thresholds[i]
            #Alert scattato: investo una percentuale del buffer
            print("### ACQUISTO ###")
            ### AAA CAMBIARE LOGICA: AUMENTARE L'INVESTIMENTO IN BASE AI MESI TRASCORSI
            number_of_purchased_stocks = int(buffer_value * thresholds_dict[thresholds[i]] / df.loc[i,"Close"])
            print("Numero di azioni acquistate: "+str(number_of_purchased_stocks))
            buffer_value = buffer_value - (number_of_purchased_stocks * df.loc[i,"Close"])
            print("Buffer updated: "+str(buffer_value))
            number_of_stocks += number_of_purchased_stocks
            pmc = ((pmc * number_of_stocks) + (df.loc[i,"Close"] * number_of_purchased_stocks))/(number_of_stocks+number_of_purchased_stocks)
            print("PMC updated: "+str(pmc))
            print("######################")
        
            ### AAA METTERE RENDIMENTO 1.5 DEL BUFFER
            ### AAA RIAZZERARE ALERT SE SONO PASSATI PIU DI X MESI DALL'ULTIMO ALERT 

In [535]:
simulation_padd(df)

PMC: 78.62000274658203
Buffer: 421.37999725341797
Numero di azioni: 1
0.0 è compreso tra -5 e 0
-0.22857181609623017 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
-0.1631996947476132 è compreso tra -5 e 0
-0.5146919950549727 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
-0.8147424615498784 è compreso tra -5 e 0
-0.8147424615498784 è compreso tra -5 e 0
-2.6949129591072865 è compreso tra -5 e 0
-1.604410627828987 è compreso tra -5 e 0
-1.604410627828987 è compreso tra -5 e 0
-2.569560608836786 è compreso tra -5 e 0
-3.7102019676149505 è compreso tra -5 e 0
PMC: 78.04500198364258
Buffer: 843.9099960327148
Numero di azioni: 2
-2.8954595060650723 è compreso tra -5 e 0
-1.2659841460069903 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
-0.39895237311649023 è compreso tra -5 e 0
-0.08727053437626774 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
0.0 è compreso tra -5 e 0
0.0 è compres

In [536]:
def simulation_pac(df):
    pmc = 0
    number_of_stocks = 0
    global monthly_investment
    for i in range(0,len(df)):
        #All'inizio del mese acquisto tot azioni quante posso permettermene, aggiorno il PMC e il buffer
        if df.iloc[i]["Date"].month != df.iloc[i-1]["Date"].month:
            number_of_stocks_purchased = int(monthly_investment / df.loc[i,"Close"])
            pmc = ((pmc * number_of_stocks) + (df.loc[i,"Close"] * number_of_stocks_purchased))/(number_of_stocks+number_of_stocks_purchased)
            number_of_stocks+=number_of_stocks_purchased
            print("PMC: "+str(pmc))
            print("Numero di azioni: "+str(number_of_stocks))

In [537]:
simulation_pac(df)

PMC: 78.62000274658203
Numero di azioni: 6
PMC: 78.04500198364258
Numero di azioni: 12
PMC: 75.56736956144634
Numero di azioni: 19
PMC: 70.94888955575449
Numero di azioni: 27
PMC: 69.96294246000402
Numero di azioni: 34
PMC: 70.1143910477801
Numero di azioni: 41
PMC: 70.32191532216173
Numero di azioni: 47
PMC: 70.62056631412145
Numero di azioni: 53
PMC: 71.12694924564683
Numero di azioni: 59
PMC: 71.46784621018628
Numero di azioni: 65
PMC: 71.68521139655313
Numero di azioni: 71
PMC: 72.36584413206421
Numero di azioni: 77
PMC: 73.0022890022002
Numero di azioni: 83
PMC: 73.69786500395013
Numero di azioni: 89
PMC: 74.3623403995595
Numero di azioni: 94
PMC: 75.1475759178701
Numero di azioni: 99
PMC: 75.89961558121901
Numero di azioni: 104
PMC: 76.5895415839799
Numero di azioni: 109
PMC: 77.38517587226734
Numero di azioni: 114
PMC: 78.15764733322528
Numero di azioni: 119
PMC: 78.96258101924774
Numero di azioni: 124
PMC: 79.6082175203072
Numero di azioni: 129
PMC: 80.25157939939571
Numero di 